Demonstrates the usage of InferExcept for overriding the inferred schema properties.

In [1]:
# Set up the Django environment for the example
from examples.django_setup import setup_django_environment

setup_django_environment()

Assuming the following Django model:

In [2]:
from typing import final

from django.db import models


@final
class SomeModel(models.Model):
    some_field = models.CharField(max_length=100, help_text="Name")

    class Meta:
        app_label = "examples"


Assuming the following Pydantic model:

In [3]:
from django2pydantic import BaseSchema, InferExcept, SchemaConfig

CUSTOM_TITLE = "Custom Title"
CUSTOM_DESCRIPTION = "Overridden Name"

class SomeSchema(BaseSchema[SomeModel]):
    """Book schema example with nested fields."""

    config = SchemaConfig[SomeModel](
        model=SomeModel,
        fields= {
            "some_field": InferExcept(description=CUSTOM_DESCRIPTION, title=CUSTOM_TITLE),
        }
    )

The following schema will be produced:

In [4]:
from rich import print_json

print_json(data=SomeSchema.model_json_schema())

{
  "properties": {
    "some_field": {
      "description": "Overridden Name",
      "maxLength": 100,
      "minLength": 1,
      "title": "Custom Title",
      "type": "string"
    }
  },
  "required": [
    "some_field"
  ],
  "title": "SomeModelSchema",
  "type": "object"
}

And as we can assert the InferExcept behavior, the generated schema will reflect the overridden properties as specified in the Pydantic model.

In [5]:
schema = SomeSchema.model_json_schema()
# Assert the title is as expected
assert schema["properties"]["some_field"]["title"] == CUSTOM_TITLE

# Assert the description is as expected
assert schema["properties"]["some_field"]["description"] == CUSTOM_DESCRIPTION